## Assignment 3 Breast Cancer Stage Classification

Breast cancer (BRCA) is the most common cancer in women. One important task to improve the survival rate of BRCA patients is identifying the cancer stage and applying different treatment strategies. We can train a model to classify cancer stages using RNA-seq of patient samples. 

Tasks:
1.	Prepare a dataset using TCGA-BRCA RNA-Seq data as features and cancer stages as labels. (Hint: you can find the processed RNA-Seq data and patient phenotype data from UCSC Xena)
2.	Applying data processing methods. (Normalization, Training-Test split, etc.)
3.	Applying three different classification estimators and optimizing the parameters through cross-validation.
4.	Comparing three estimators by evaluating the performance on the test dataset.
5.	Applying feature selection to improve performance.


### Task 1

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scikitplot as skplt

In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle

In [68]:
reads = pd.read_csv('TCGA-BRCA.htseq_fpkm.tsv', sep='\t', header=0)
label = pd.read_csv('TCGA-BRCA.GDC_phenotype.tsv', sep='\t', header=0)

In [67]:
id_ = pd.read_excel('oncotarget.xlsx', sheet_name='Sheet1', header=1, index_col=0)
# id_.reset_index(inplace=True, drop=True)
id_.rename(columns={'Feature name':'gene'}, inplace=True)
id_ = id_.iloc[:4970,:]
id_

,gene,MI value
Order,,
1,ENSG00000155657,0.41616
2,ENSG00000008988,0.40741
3,ENSG00000177600,0.40519
4,ENSG00000211772,0.39637
5,ENSG00000168028,0.39332
...,...,...
4966,ENSG00000028277,0.09720
4967,ENSG00000169242,0.09720
4968,ENSG00000069424,0.09720


In [69]:
reads[['Ensembl_ID']].to_csv('ucsc_id.csv', index=False)

In [49]:
reads['Ensembl_ID'] = reads['Ensembl_ID'].str.split('.').str[0]

In [50]:
reads2 = reads[reads['Ensembl_ID'].isin(id_['gene'])]
reads2

,Ensembl_ID,TCGA-E9-A1NI-01A,TCGA-A1-A0SP-01A,TCGA-BH-A1EU-11A,TCGA-A8-A06X-01A,TCGA-E2-A14T-01A,TCGA-AC-A8OS-01A,TCGA-A8-A09K-01A,TCGA-OL-A5RY-01A,TCGA-BH-A0DG-01A,...,TCGA-BH-A0DT-11A,TCGA-E9-A1R0-01A,TCGA-BH-A0B6-01A,TCGA-B6-A0RN-01A,TCGA-A8-A09W-01A,TCGA-EW-A1P3-01A,TCGA-A7-A13F-11A,TCGA-A2-A0T6-01A,TCGA-A7-A5ZW-01A,TCGA-BH-A203-01A
1,ENSG00000270112,0.019573,0.004701,0.016302,0.000000,0.000000,0.000000,0.005787,0.000000,0.000000,...,0.000000,0.003879,0.010047,0.000000,0.010919,0.000000,0.000000,0.000000,0.005858,0.008704
2,ENSG00000167578,2.235898,1.863334,1.704753,1.947481,2.734690,2.397119,2.337327,2.256976,1.968791,...,1.928209,2.004722,2.971522,1.841232,1.185122,2.861514,1.626213,1.692995,2.051916,2.197365
8,ENSG00000198242,7.705589,6.835508,7.125310,7.259318,7.643035,6.474977,7.360486,7.620826,7.195859,...,7.209058,7.973670,7.141150,7.380147,7.497508,7.673599,7.439460,7.138577,7.446467,8.515192
15,ENSG00000172137,2.438029,0.649640,3.749056,0.175693,2.229444,3.240255,1.061342,4.662264,2.233367,...,2.616483,3.941888,0.336239,1.449140,0.742809,1.516158,5.997365,3.348625,1.753505,1.702475
16,ENSG00000167700,4.733326,3.973001,2.197980,4.481979,4.674835,3.885666,4.426910,3.603154,3.221832,...,2.672013,3.779761,4.574800,4.156321,3.160879,4.346738,2.796725,3.051859,3.409768,1.887785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60436,ENSG00000162999,0.636671,0.554912,1.134340,0.603737,0.775687,0.640493,1.138300,0.606646,0.736084,...,1.184874,0.847677,0.561274,1.048010,0.681519,1.070615,0.826445,0.757035,0.996614,0.583245
60438,ENSG00000102265,8.471759,6.812608,6.423011,7.771150,7.748566,7.267486,6.449583,8.404308,7.509621,...,6.675153,8.104923,7.579762,7.117155,6.540043,8.796616,6.260510,7.096658,7.633693,7.364423
60450,ENSG00000066044,3.721245,4.459218,3.404217,4.491241,3.781642,3.442338,3.929391,4.035544,3.959681,...,3.608936,4.061367,4.086652,3.941680,3.912703,3.951310,3.214453,3.686299,3.605157,3.785127
60472,ENSG00000009694,0.098232,0.011167,0.322213,0.016356,0.034593,0.158967,0.102133,0.040433,0.139216,...,0.385508,0.054448,0.023804,0.025628,0.122912,0.021983,0.401469,0.334910,0.199790,0.034231


In [34]:
class classifier(object):
    def __init__(self, reads, label):
        self.reads = reads
        self.label = label
        self.data = None

    def preprossessing(self):
        '''Merge the reads and label dataframes into a single dataframe'''
        reads = self.reads.set_index('Ensembl_ID').T
        reads.reset_index(inplace=True)
        reads.rename(columns={'index':'sample_ID'}, inplace=True)

        self.label = label.loc[:, ['submitter_id.samples', 'tumor_stage.diagnoses']].rename(columns={'submitter_id.samples':'sample_ID', 'tumor_stage.diagnoses': 'diagnosis'})
        print('label 1', self.label.shape)
        self.label.dropna(inplace=True)
        print('label (drop NA)', self.label.shape)
        self.label = self.label.query('diagnosis != "not reported"') # exclude samples with no diagnosis
        # self.label = self.label.loc[:,['sample_ID','diagnosis']] # extract useful info
        # label2.dropna(inplace=True)
        print('label (drop "not reported")', self.label.shape)
        print('The shape of the whole data frame:', reads.shape)
        data = pd.merge(reads, self.label, on='sample_ID', how='inner') # merge two dataframes
        print('The merged data shape: ', data.shape)
        self.data = data
        # when the whole process is finished, the return value can be None
        return data

    def dimen_reduction(self):
        pass

    def train_test_split(self):
        '''First convert the label to binary numerical values, then split the data into training and testing sets;
        A label dictionary is created to map the numerical values back to the original labels
        '''
        # labels = self.data['sample_type.samples']
        # y = preprocessing.OneHotEncoder().fit_transform(labels.values.reshape(-1,1)).toarray()
        self.y = self.data['diagnosis']
        self.X = self.data.drop(['sample_ID', 'diagnosis'], axis=1)
        # self.label_dict = {0:'Metastatic', 1:'Primary Tumor', 2:'Solid Tissue Normal'}
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.3, random_state=42)

### Action codes

In [51]:
A3 = classifier(reads2, label)
data = A3.preprossessing()
A3.train_test_split()

label 1 (1284, 2)
label (drop NA) (1282, 2)
label (drop "not reported") (1270, 2)
The shape of the whole data frame: (1217, 4891)
The merged data shape:  (1204, 4892)


## Below are draft

In [52]:
A3.data.diagnosis.value_counts()

stage iia     397
stage iib     290
stage iiia    172
stage i       105
stage ia       91
stage iiic     70
stage iiib     31
stage iv       22
stage x        12
stage ib        6
stage ii        6
stage iii       2
Name: diagnosis, dtype: int64

### SVM

In [66]:
# construct multi-class SVM classifier
model = SVC(kernel='linear', gamma="auto")
model.fit(A3.X_train, A3.y_train)
y_pred = model.predict(A3.X_test)
print('Accuracy: ', accuracy_score(A3.y_test, y_pred))
# print('Classification report: \n', classification_report(A3.y_test, y_pred))

Accuracy:  0.287292817679558


### Use random forest

In [58]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier()
model3.fit(A3.X_train, A3.y_train)
y_pred3 = model3.predict(A3.X_test)
print('Accuracy: ', accuracy_score(A3.y_test, y_pred3))

Accuracy:  0.2900552486187845


In [61]:
# search the best parameters
param_grid = {'max_depth': [3, 5, 7, 9],'n_estimators': [50, 100], 'max_features': [5, 10, 20, 30]}
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy', verbose=3)
grid.fit(A3.X_train, A3.y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/damarisdeng/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END max_depth=3, max_features=5, n_estimators=50;, score=0.337 total time=   0.3s
[CV 2/5] END max_depth=3, max_features=5, n_estimators=50;, score=0.337 total time=   0.2s
[CV 3/5] END max_depth=3, max_features=5, n_estimators=50;, score=0.345 total time=   0.2s
[CV 4/5] END max_depth=3, max_features=5, n_estimators=50;, score=0.351 total time=   0.2s
[CV 5/5] END max_depth=3, max_features=5, n_estimators=50;, score=0.339 total time=   0.2s
[CV 1/5] END max_depth=3, max_features=5, n_estimators=100;, score=0.337 total time=   0.3s
[CV 2/5] END max_depth=3, max_features=5, n_estimators=100;, score=0.337 total time=   0.3s
[CV 3/5] END max_depth=3, max_features=5, n_estimators=100;, score=0.345 total time=   0.3s
[CV 4/5] END max_depth=3, max_features=5, n_estimators=100;, score=0.339 total time=   0.3s
[CV 5/5] END max_depth=3, max_features=5, n_estimators=100;, score=0.345 total time=   0.3s
[CV 1/5] END max_depth=3, max_features=10, n_estimators=50;, score=0.337 total time= 

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 7, 9],
                         'max_features': [5, 10, 20, 30],
                         'n_estimators': [50, 100]},
             scoring='accuracy', verbose=3)

In [62]:
grid.best_params_, grid.best_score_

{'max_depth': 7, 'max_features': 20, 'n_estimators': 50}

In [65]:
grid.cv_results_

{'mean_fit_time': array([0.1826561 , 0.29092736, 0.21086411, 0.41257081, 0.31799407,
        0.55869017, 0.38221231, 0.81318889, 0.21136456, 0.36974773,
        0.28411822, 0.54925227, 0.40161858, 0.76813455, 0.54208841,
        1.02760296, 0.21904163, 0.39834914, 0.3177762 , 0.61494775,
        0.5024653 , 0.96668673, 0.70608997, 1.43477364, 0.31327119,
        0.46985445, 0.36743464, 0.64126163, 0.54867039, 1.15439243,
        0.79831204, 1.46180611]),
 'std_fit_time': array([0.01974514, 0.00323267, 0.0017089 , 0.03852648, 0.01876227,
        0.01316651, 0.01155651, 0.07054085, 0.01407888, 0.02041988,
        0.00326527, 0.0210055 , 0.00685625, 0.01444116, 0.0077284 ,
        0.02946802, 0.00504087, 0.00979048, 0.01641859, 0.07230447,
        0.04526419, 0.033982  , 0.02552722, 0.08821324, 0.10115993,
        0.02624107, 0.02881167, 0.01312623, 0.01340393, 0.11270053,
        0.03245287, 0.02007705]),
 'mean_score_time': array([0.04486427, 0.04898858, 0.04504981, 0.05971379, 0.046372

In [63]:
grid.predict(A3.X_test)
print('Accuracy: ', accuracy_score(A3.y_test, grid.predict(A3.X_test)))

Accuracy:  0.287292817679558


Use decision tree to predict based on the selected features